In [1]:
def convertToBFS(line):
    fields = line.split()
    heroID = int(fields[0])
    connections = []
    for connection in fields[1:]:
        connections.append(int(connection))
    
    color ="WHITE"
    distance = 9999
    
    if (heroID == startCharacterID):
        color = "GRAY"
        distance = 0
    
    return(heroID, (connections, distance, color))

In [2]:
def createStartingRDD():
    inputFile = sc.textFile('Marvel-Graph.txt')
    return inputFile.map(convertToBFS)

In [3]:
def bfsMap(node):
    characterID = node[0]
    data = node[1]
    connections = data[0]
    distance = data[1]
    color = data[2]
    
    results = []
    
    #if this node expanded 
    if(color == 'GRAY'):
        for connection in connections:
            newCharacterID = connection            
            newDistance = distance + 1
            newColor = 'GRAY'
            if (targetCharacterID == connection):
                hitCounter.add(1)
                
            newEntry = (newCharacterID, ([], newDistance, newColor))
            results.append(newEntry)
            
        #we have processed node so color it BLACK
        color = 'BLACK'
            
    #Emint the input node to keep track of it
    results.append((characterID,(connections, distance, color)))
    return results   
    

In [4]:
def bfsReduce(data1, data2):
    edges1 = data1[0]
    edges2 = data2[0]
    distance1 = data1[1]
    distance2 = data2[1]
    color1 = data1[2]
    color2 = data2[2]
    
    distance = 9999
    color =  color1
    edges = []
    
    # see if one is original node with connections
    # preserve them
    if (len(edges1) > 0):
        edges.extend(edges1)
    if (len(edges2) > 0):
        edges.extend(edges2)
        
    # maintain minimum distacne
    if (distance1 < distance):
        distance = distance1
        
    if (distance2 < distance):
        distance = distance2
    
    #preserve the darkest color
    if (color1 == 'WHITE' and (color2 == 'GRAY' or color2 == 'BLACK')):
        color = color2
     
    if (color1 == 'GRAY' and  color2 == 'BLACK'):
        color = color2
        
    if (color2 == 'WHITE' and (color1 == 'GRAY' or color1 == 'BLACK')):
        color = color1
     
    if (color2 == 'GRAY' and  color1 == 'BLACK'):
        color = color1
    
    return (edges, distance, color)    

In [6]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('DegreesOfSeperations')
sc = SparkContext(conf = conf)

startCharacterID = 5306 #spiderman
targetCharacterID = 14 #ADAM
hitCounter = sc.accumulator(0)

iterationRdd = createStartingRDD()

for iteration in range(0, 15):
    print('Running BFS ireations# {}'.format(iteration+1))
    
    # Create new vertices as needed to darken or reduce distances in the
    # reduce stage. If we encounter the node we're looking for as a GRAY
    # node, increment our accumulator to signal that we're done.
    mapped = iterationRdd.flatMap(bfsMap)
    
    # Note that mapped.count() action here forces the RDD to be evaluated, and
    # that's the only reason our accumulator is actually updated.
    print('Processing  {} values'.format(mapped.count()))
    
    if (hitCounter.value > 0):
        print("Hit the target character! From {} different direction(s)."\
             .format(hitCounter.value))
        break        
        
    # Reducer combines data for each character ID, preserving the darkest
    # color and shortest path. 
    iterationRdd = mapped.reduceByKey(bfsReduce)
sc.stop()

Running BFS ireations# 1
Processing  8330 values
Running BFS ireations# 2
Processing  220615 values
Hit the target character! From 1 different direction(s).
